# MDP based Recommender System

In [283]:
# dependencies
import pymongo
import pandas as pd
import numpy as np
from queue import  Queue
from copy import  copy
from matplotlib import pyplot as plt

In [6]:
cl = pymongo.MongoClient()
db = cl.steam
print("Collections")
for i in db.collection_names():
    print("   - "+i)

Collections
   - appList
   - appDetail
   - userOwns
   - userAchieve
   - userSummary
   - userBan
   - appReview


## Q: is userOwns-data ordered with user-purchaed time ?
## Q: 유저게임소유데이터( userOwns-data) 는 유저 구매순으로 정렬되어 있는가?

In [7]:
userOwnsMoreThan1games = db.userOwns.find({"game_count":{"$gte":1}})

In [194]:
db.userBan.count()

390546

In [8]:
uomt1Size = userOwnsMoreThan1games.count()
x1_x2_y = []
for _ in range(uomt1Size):
    games = userOwnsMoreThan1games.next()['games']
    x1 = -1
    for x in map(lambda x:x["appid"],games):
        if x1 == -1:
            x1 = x
            continue
        x1_x2_y.append([x1,x,x>x1])
        x1 = x

In [9]:
x1_x2_y = pd.DataFrame(x1_x2_y,columns= ["x1","x2","y"])

In [10]:
p_x2_gt_x1  = x1_x2_y[x1_x2_y.y == True].size/x1_x2_y.size
p_x1_gt_x2  = x1_x2_y[x1_x2_y.y == False].size/x1_x2_y.size
print("probability x2 is greater than x1  is {0}".format(p_x2_gt_x1))
print("probability x1 is greater than x2  is {0}".format(p_x1_gt_x2))

probability x2 is greater than x1  is 0.7362341767571652
probability x1 is greater than x2  is 0.2637658232428349


#### $<X_1, X_2, .... X_t> $ 이 유저가 소유한 게임들의 시퀀스 일때 &nbsp; $$P(X_t > X_{t-1}) = 0.73$$ $$P(X_t < X_{t-1}) = 0.26$$

#### H1: userOwns game list is ordered with release date (ascending). <- 기각 <br> H2: userOwns game list is ordered with release date (descending). <- 기각 <br> H3: userOwns game list is ordered with random. <- 보류 <br> H4: userOwns game list is ordered with purchased date. <- 보류

#### 유저 게임 소유 데이터는 구매순으로 나열되어있다고 볼 수 있다.
#### 데이터셋에 시간순서 $t$ 가 존재하므로 MDP (Markov Decision Process) 문제로 확대 가능 하다.

In [11]:
usersHaveMore10Games = db.userOwns.find({"game_count":{"$gte":10}}) # 10개 이상의 게임을 구매한 고객
print("{0} 명의 유저가 10개이상의 게임을 구매함".format(usersHaveMore10Games.count())) # 12779 명

12779 명의 유저가 10개이상의 게임을 구매함


In [12]:
allGames = x1_x2_y.x1.append(x1_x2_y.x2).unique() 
print("{:<5d} : 모든 게임의 수(샘플 {:5d} 명 유저 구매 기준)".format(len(allGames),userOwnsMoreThan1games.count()))

topGames = x1_x2_y.x1.value_counts()[x1_x2_y.x1.value_counts() > 1000].index # 441개
print("{:<5d} :  1000명 이상의 구매자가 있는 게임 수(샘플 {:5d} 명 유저 구매 기준)".format(topGames.size,userOwnsMoreThan1games.count()))

restGames = list(set(allGames) - set(topGames) )
print("{:<5d} :  1000명 이하의 구매자가 있는 게임 수(샘플 {:5d} 명 유저 구매 기준)".format(len(restGames),userOwnsMoreThan1games.count()))

23252 : 모든 게임의 수(샘플 13642 명 유저 구매 기준)
441   :  1000명 이상의 구매자가 있는 게임 수(샘플 13642 명 유저 구매 기준)
22811 :  1000명 이하의 구매자가 있는 게임 수(샘플 13642 명 유저 구매 기준)


In [13]:
top_rest_dic = {}   # dictionary to sort
for i in topGames:
    top_rest_dic.update({i:True})
for i in restGames:
    top_rest_dic.update({i:False})

In [248]:
class State:
    def __init__(self,s_arr):
        self.state_arr = s_arr
    def __str__(self):
        return "State(["+",".join(map(lambda x:str(x),self.state_arr))+"])"
    def __hash__(self):
        return hash(self.__repr__())
    def __repr__(self):
        return self.__str__()
    def __eq__(self,other):
        return self.state_arr == other.state_arr
    def getAction(self):
        return self.state_arr[-1]
    def step(self,action):
        return State(self.state_arr[1:] + [action])

In [249]:
def queueProcess(x,q,g,g_rev):
    nullState = State([-1])
    if q.full():
        state_1 = g["state_1"]
        state = State(list(q.queue))
        if state_1 != nullState:
            try:
                g[state_1][state] += 1
            except:
                try:
                    g[state_1].update({state:1})
                except:
                    g.update({state_1:{}})
                    g[state_1].update({state:1})
                    
            try:
                g[state]
            except:
                g.update({state:{}})
                
                
            try:
                g_rev[state][state_1] += 1
            except:
                try:
                    g_rev[state].update({state_1:1})
                except:
                    g_rev.update({state:{}})
                    g_rev[state].update({state_1:1})
                
        g["state_1"] = state
        q.get()
        q.put(x)
    else:
        q.put(x)
    return q,g,g_rev

The form of DAG, $g_{t,t+n} $ is {$s_t$ : {$s_{t+1}$: $count \ of \ <s_t, s_{t+1}>$ , ....}}

In [250]:
g1 = {}
g1_2 = {}
g1_rev = {}
g1_2_rev = {}
userOwnsMoreThan1games = db.userOwns.find({"game_count":{"$gt":1}})
uomt1Size = userOwnsMoreThan1games.count()
for _ in range(uomt1Size-1):
    games = userOwnsMoreThan1games.next()['games']
    q1 = Queue(maxsize=1)
    q1_2 = Queue(maxsize=2)
    g1["state_1"]= State([-1])
    g1_2["state_1"]= State([-1])
    for x in map(lambda x:x["appid"],games):
        if not top_rest_dic[x]:
            continue
        q1,g1,g1_rev =  queueProcess(x,q1,g1,g1_rev)
        q1_2,g1_2,g1_2_rev =  queueProcess(x,q1_2,g1_2,g1_2_rev)

In [251]:
g1_2

{State([393380,225540]): {State([225540,208650]): 4,
  State([225540,236150]): 1,
  State([225540,268500]): 2,
  State([225540,311210]): 5,
  State([225540,359550]): 14,
  State([225540,391220]): 3,
  State([225540,417860]): 3,
  State([225540,431240]): 3,
  State([225540,431960]): 3,
  State([225540,438100]): 2,
  State([225540,444090]): 1,
  State([225540,477160]): 1,
  State([225540,489830]): 1,
  State([225540,578080]): 1},
 State([381210,379720]): {State([379720,273350]): 1,
  State([379720,287700]): 1,
  State([379720,289070]): 1,
  State([379720,292030]): 1,
  State([379720,337000]): 1,
  State([379720,34270]): 1,
  State([379720,359550]): 3,
  State([379720,360430]): 3,
  State([379720,364360]): 2,
  State([379720,377160]): 2,
  State([379720,403640]): 2,
  State([379720,475150]): 2,
  State([379720,477160]): 6,
  State([379720,48000]): 3,
  State([379720,489520]): 1,
  State([379720,489830]): 2,
  State([379720,50130]): 2,
  State([379720,504370]): 4,
  State([379720,530620]):

In [252]:
import json
with open('g1_2.txt', 'w') as outfile:
    json.dump(str(g1_2), outfile)
with open('g1_2_rev.txt', 'w') as outfile:
    json.dump(str(g1_2_rev), outfile)

### Defining the MDP
the **state** is **sequence of games** recent t ~ t-n bought
$$S_t = <g_t> \ \ or$$ 

$$S_t = <g_{t-1}, g_t>\ \ or$$

$$S_t = <g_{t-2}, g_{t-1}, g_t>\ \ or$$

$$S_t = <g_{t-3},g_{t-2}, g_{t-1}, g_t>$$


the **action** of MDP correspond to a **recommendation of an game** (action 은 game 추천) <br>
the **rewards** of MDP correspond to a $r(s_{t+1},a,s_t)= \{w_1 \times count( \ \cdot \ ,s_{t+1})\} \  + \ \{w_2 \times count(s_{t+1}, \ \cdot \ )\} \ ,  \ \ where \ \ w_1 + w_2 = 1 $ 

transition probability &nbsp; $tr_{MDP}(<x_1, x_2, x_3>, \ x', \ < x_2, x_3,x''>)$ <br>
meaning that $P( s_{t+1} \ | \ s_t,a_t)$ &nbsp; $where$  &nbsp; $s_t = <x_1, x_2, x_3> $ , &nbsp; $s_{t+1} = <x_2, x_3,x''> $ , $a_t = x' $
<br><br>
assumptions of initial transition probability

- A recommendation(action) increases the probability that a user will buy an game.
    - This probability is proportional to the probability taht the user will buy this game in the absence of recommendation 
    - This assumption is made by most CF models dealing with e-commerce sites.
    - we denote the proportionality constant  by  $\alpha > 1$

- The probability that a user will buy an game that was not recommened is lower than the probability that the agent will buy without recommendation.
    - we denote the proportionality constant  by  $\beta < 1$
    
    
$$tr_{MDP}(<x_1, x_2, x_3>, \ x', \ < x_2, x_3,x'>) = \alpha \cdot tr_{MC}(<x_1, x_2, x_3>, \ < x_2, x_3,x'>) $$

$$tr_{MDP}(<x_1, x_2, x_3>, \ x', \ < x_2, x_3,x''>) = \beta \cdot tr_{MC}(<x_1, x_2, x_3>, \ < x_2, x_3,x''>) \ , \ \  x'' \neq x'$$

### Bellman Equation
$$v_\pi(s) = \sum\limits_{a}\pi(a|s)\sum\limits_{s'}tr_{MDP}[r+\gamma v_\pi(s')]$$

In [253]:
rewards_g1 = {}
rewards_g1_2 = {}
#rewards_g1_3 = {}
g1_states_arr = list(g1.keys())
g1_2_states_arr = list(g1_2.keys())
#g1_3_states_arr = list(g1_3.keys())

In [254]:
def getRewardHash(g,g_rev):
    r_g = {}
    g_states_arr = list(g.keys())
    for i in g_states_arr:
        if type(i)  != State:
            continue;
        try:
            popularScore = sum(g[i].values())
        except:
            popularScore = 0
            
        try:
            afterEffectScore = sum(g_rev[i].values())
        except:
            afterEffectScore = 0        
            
        reward_i = 0.5*popularScore+0.5* afterEffectScore
        r_g.update({i:reward_i})
    return r_g

In [255]:
rewards_g1 = getRewardHash(g1,g1_rev)
rewards_g1_2 = getRewardHash(g1_2,g1_2_rev)
rewards_g1_3 = getRewardHash(g1_3,g1_3_rev)

In [256]:
def get_transProbMat(g):
    g_states =  list(g.keys())
    s_size = len(g_states)
    state_index = {s:idx for idx,s in enumerate(g_states)}
    transProbMat = np.zeros([s_size,s_size])  # transition probability 
    for s in g_states:
        if type(s) != State:
            continue
        g_s_ss = g[s]
        g_s_ss_sum = sum(g_s_ss.values())
        idxI = state_index[s]
        for ss in list(g_s_ss.keys()):
            idxJ = state_index[ss]
            transProbMat[idxI][idxJ] = g_s_ss[ss]/g_s_ss_sum
    return transProbMat,state_index

### Solving the MDP using Policy Iteration

In [257]:
class Env:
    def __init__(self,g_DAG,reward_dict,transProbMat,state_index):
        self.g_DAG = g_DAG
        self.reward_dict = reward_dict
        self.transProbMat = transProbMat
        self.state_index = state_index
    
    def get_reward(self,  state, action):
        next_state = state.step(action)
        return self.reward_dict[next_state]
    
    def get_transition_prob(self, state, action):
        stateIDX = state_index[state]
        next_stateIDX = state_index[state.step(action)]
        return self.transProbMat[stateIDX][next_stateIDX]
    
    def get_transition_prob_arr(self, state):
        stateIDX = state_index[state]
        return self.transProbMat[stateIDX]
    
    def get_possibleActions(self,state):
        next_states = list(self.g_DAG[state].keys())
        return [i.getAction() for i in next_states]
    
    def get_all_states(self):
        return list(self.g_DAG.keys())

In [258]:
class PolicyIteration:
    transi_Alpha = 1.5
    
    def __init__(self, env):
        self.env = env
        self.value_hash = {state:0.0 for state in env.get_all_states()}
        self.policy_hash = {state:{a:0.0 for a in env.get_possibleActions(state)} for state in env.get_all_states()}
        self.fill_policy_hash()
        self.discount_factor = 0.9
    
    def fill_policy_hash(self):
        for state in self.policy_hash:
            next_state_len = len(self.policy_hash[state])
            if next_state_len == 0:
                continue
            for a in self.policy_hash[state]:
                self.policy_hash[state][a] = 1/next_state_len
    
    def transition_prob(self,state,action,state_action_probList):
        index = env.state_index[state.step(action)]
        state_action_probList[index] = min(self.transi_Alpha * state_action_probList[index],1)
        actionProb = copy(state_action_probList[index])
        if state_action_probList[index] == 1 or (sum(state_action_probList)-state_action_probList[index]) < 0.01:
            beta = 0
        else:
            beta = (1 - state_action_probList[index])/(sum(state_action_probList)-state_action_probList[index])
        state_action_probList = state_action_probList*beta
        state_action_probList[index] = actionProb
        return state_action_probList
        
    def policy_evaluation(self):
        next_value_hash = {state:0.0 for state in env.get_all_states()}
        
        # Bellman Expectation Equation for the every states
        for state in self.env.get_all_states():
            value = 0.0
            stateProbList = self.env.get_transition_prob_arr(state)
            for action in self.env.get_possibleActions(state):
                
                state_action_probList = copy(stateProbList)
                state_action_probList = self.transition_prob(state,action,state_action_probList)
                policyProb = self.get_policy(state)[action]
                reward = self.env.get_reward(state,action)
                nextStates = list(env.g_DAG[state].keys())
                for next_state in nextStates:
                    next_value = self.get_value(next_state)
                    value += (policyProb *(reward + state_action_probList[env.state_index[next_state]]*self.discount_factor * next_value))
            next_value_hash[state] = round(value, 2)

        self.value_hash = next_value_hash

    def policy_improvement(self):
        next_policy = self.policy_hash
        for state in self.env.get_all_states():
            value = -99999
            max_actions = []
            result = {}  # initialize the policy

            # for every actions, calculate
            # [reward + (discount factor) * transition probability*(next state value function)]
            stateProbList = self.env.get_transition_prob_arr(state)
            for action in self.env.get_possibleActions(state):
                next_policy[state][action] = 0.0
                next_state = state.step(action)
                reward = self.env.get_reward(state,action)
                next_value = self.get_value(next_state)
                temp = reward + self.discount_factor *next_value

                # We normally can't pick multiple actions in greedy policy.
                # but here we allow multiple actions with same max values
                if temp == value:
                    max_actions.append(action)
                elif temp > value:
                    value = temp
                    max_actions.clear()
                    max_actions.append(action)

            # probability of action
            if len(max_actions) != 0:
                prob = 1 / len(max_actions)

                for actions in max_actions:
                    next_policy[state][actions] = prob


        self.policy_table = next_policy

    # get action according to the current policy
    def get_action(self, state):
        random_pick = random.randrange(100) / 100

        policy = self.get_policy(state)
        policy_sum = 0.0
        # return the action in the index
        for index, value in enumerate(policy):
            policy_sum += value
            if random_pick < policy_sum:
                return index

    # get policy of specific state
    def get_policy(self, state):
        return self.policy_hash[state]

    def get_value(self, state):
        return round(self.value_hash[state], 2)


In [259]:
g1.pop('state_1', None)
g1_2.pop('state_1', None)
#g1_3.pop('state_1', None)
transProbMat,state_index = get_transProbMat(g1_2)
env = Env(g1_2,rewards_g1_2,transProbMat,state_index)
policy_iteration = PolicyIteration(env)


In [261]:
for _ in range(1):
    policy_iteration.policy_evaluation()
#policy_iteration.value_hash

In [262]:
policy_iteration.value_hash

{State([393380,225540]): 0.0,
 State([239140,355840]): 0.0,
 State([381210,379720]): 0.0,
 State([223530,333930]): 0.0,
 State([35420,400]): 0.0,
 State([200710,4760]): 0.0,
 State([252950,225540]): 0.0,
 State([380600,57300]): 0.0,
 State([105450,218640]): 0.0,
 State([246620,19900]): 0.0,
 State([245170,235540]): 0.0,
 State([346900,70]): 0.0,
 State([211160,205950]): 0.0,
 State([42710,91310]): 0.0,
 State([250400,38400]): 0.0,
 State([8190,291550]): 0.0,
 State([289070,286160]): 0.0,
 State([12250,17460]): 0.0,
 State([200710,1250]): 0.0,
 State([211160,203750]): 0.0,
 State([43110,202090]): 0.0,
 State([403640,299360]): 0.0,
 State([239140,737010]): 0.0,
 State([269210,201700]): 0.0,
 State([225260,218620]): 0.0,
 State([248570,253710]): 0.0,
 State([223470,431960]): 0.0,
 State([8870,22330]): 0.0,
 State([12250,203770]): 0.0,
 State([57300,204100]): 0.0,
 State([20920,35700]): 0.0,
 State([202750,50300]): 0.0,
 State([312990,333930]): 0.0,
 State([40800,19900]): 0.0,
 State([2092

In [102]:
policy_iteration.policy_improvement()

In [104]:
#policy_iteration.policy_hash

In [108]:
state_same = []
same = 0
notsame = 0
for i in policy_iteration.policy_hash.keys():
    for j in policy_iteration.policy_hash[i]:
        if policy_iteration.policy_hash[i][j] > 0.0:
            if g1_2[i][i.step(j)] == max(g1_2[i].values()):
                state_same.append((i,1))
                same += 1
            else:
                state_same.append((i,0))
                notsame +=1
            break
                
            

In [111]:
same,notsame

(31932, 6371)

In [113]:
arr = [[1,2,3],[3,4,5]]

In [115]:
tt = copy(arr[0])

In [117]:
tt[0] = 10

In [118]:
tt

[10, 2, 3]

In [119]:
arr

[[1, 2, 3], [3, 4, 5]]

# $<g_1, \ g_2, \ g_{3}, \ ... \ g_{n} >$

In [188]:
sum(g1[State([578080])].values())

6503

In [189]:
sum(g1_rev[State([578080])].values())

8647

# $$\pi(a \ | \ s)$$


In [197]:
len(g1_2.keys())

38642

In [198]:
ls

Bluehole.png    analysis.ipynb  g1_2_rev.txt    pubg.key
Bluehole.svg    g1_2.txt        game.txt


In [205]:
f = open("value_hash.txt","r")
value_hash = f.
f.close()
value_hash_ = value_hash[0]
value_hash = eval(value_hash_[1:-1])
value_hash

In [221]:
g1_2

{State([393380,225540]): {State([225540,208650]): 4,
  State([225540,236150]): 1,
  State([225540,268500]): 2,
  State([225540,311210]): 5,
  State([225540,359550]): 14,
  State([225540,391220]): 3,
  State([225540,417860]): 3,
  State([225540,431240]): 3,
  State([225540,431960]): 3,
  State([225540,438100]): 2,
  State([225540,444090]): 1,
  State([225540,477160]): 1,
  State([225540,489830]): 1,
  State([225540,578080]): 1},
 State([381210,379720]): {State([379720,273350]): 1,
  State([379720,287700]): 1,
  State([379720,289070]): 1,
  State([379720,292030]): 1,
  State([379720,337000]): 1,
  State([379720,34270]): 1,
  State([379720,359550]): 3,
  State([379720,360430]): 3,
  State([379720,364360]): 2,
  State([379720,377160]): 2,
  State([379720,403640]): 2,
  State([379720,475150]): 2,
  State([379720,477160]): 6,
  State([379720,48000]): 3,
  State([379720,489520]): 1,
  State([379720,489830]): 2,
  State([379720,50130]): 2,
  State([379720,504370]): 4,
  State([379720,530620]):

In [280]:
scores = []
for listSize in range(1,11):
    userOwnsMoreThan1games = db.userOwns.find({"game_count":{"$gt":1}})
    userOwnsMoreThan1games
    uomt1Size = userOwnsMoreThan1games.count()
    record_sucess_fail = []
    for _ in range(uomt1Size-1):
        games = userOwnsMoreThan1games.next()['games']
        appid_1 = -1
        gameIds = list(map(lambda x:x["appid"],games))
        maxLen = len(gameIds)
        for idx,x in enumerate(gameIds):
            if not top_rest_dic[x]:
                continue
            appid = x
            if appid_1 == -1:
                appid_1 = appid
                continue
            state = State([appid_1,appid])
            appid_1 = appid
            try:
                actions = env.get_possibleActions(state)
            except:
                continue
            nextState_values = []
            for action in actions:
                next_state = state.step(action)
                next_state_val = value_hash[next_state]
                nextState_values.append((next_state,next_state_val))
            nextState_values = sorted(nextState_values,key=lambda x:x[1],reverse=True)
            if maxLen == idx +1 :
                break
            realNext = state.step(gameIds[idx + 1])
            if realNext in [i for i,_ in nextState_values[:min(len(nextState_values),listSize)]]:
                record_sucess_fail.append(1)
            else:
                record_sucess_fail.append(0)
    scores.append(sum(record_sucess_fail)/len(record_sucess_fail))



In [287]:
plt.plot(np.array(scores)*100)
plt.title("Recommend Score")
plt.ylabel('RC score')
plt.xlabel("recommend list size")
plt.show()
scores

[0.1577994608322268,
 0.21630781962156925,
 0.25604347119336784,
 0.28726516963518567,
 0.3131426894634288,
 0.3392323720215414,
 0.35825946340773457,
 0.37984622951996333,
 0.39581121105863926,
 0.41124103586077904]